**Antes que nada correr esta celda para poder usar julia**

In [ ]:
# Installation cell
%%capture
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget -q 'https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.1-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'
echo 'Done'

#Multiple Dispatch

Es un concepto muy importante en Julia. Ahora ¿Que pingo significa esto ura?. Ya te explico chango, la cosa es así.

Nosotros cuando hablamos de un envio simple estamos hablando de un proceso en donde elegimos un método diferente o versión de una función dependiendo del tipo de entrada

Y el envio multiple (*Multiple dispatch*) significa que miramos todos los tipos de entrada para todos los argumentos de la función para elegir que método o versión vamos a llamar

In [ ]:
#Defino un numero complejo
cc = 3 +4im #Para definir un imaginario se pone im dsp del num

3 + 4im

In [ ]:
#Para poder ver que es lo que realmente hace Julia voy a usar un macro
@which cc + cc

+(z::Complex, w::Complex) in Base at complex.jl:275

Lo que me dice lo de arriba es que se usa un método que recibe dos numeros complejos y los suma.

Ahora vamos a ver que pasa si sumamos un numero real

In [ ]:
@which cc + 4 

+(z::Complex, x::Real) in Base at complex.jl:313

Vemos que ahora llama a otro método, entonces esto quiere decir que entre todas las funciones definidas para la suma llama especificamente a aquellas que reciba ciertos parametros esperados

In [ ]:
#Vuelvo a probar pero invirtiendo
@which 4 + cc

+(x::Real, z::Complex) in Base at complex.jl:312

In [ ]:
#Quiero ver cuantos métodos de "+" hay
+

+ (generic function with 190 methods)

**Nota importante**, cada vez que usamos este macro nos da un link a la implementación, tenganlo en cuenta

# Multi Threading

Este modelo de paralelismo tiene muchas propiedades maravillosas. Lo vemos como algo análogo a la recolección de basura: con GC, usted asigna objetos libremente sin preocuparse por cuándo y cómo se liberan. Con el paralelismo de tareas, genera tareas libremente (potencialmente millones de ellas) sin preocuparse por dónde se ejecutan.

In [ ]:
import Distributed
using Distributed

In [4]:
nprocs() # Para ver la cantidad de procesos

1

In [5]:
workers() # Para ver la cantidad de workers

1-element Vector{Int64}:
 1

In [6]:
addprocs(1) # Para agregar más procesos

1-element Vector{Int64}:
 2

In [7]:
nprocs() # Para ver la cantidad de procesos

2

In [8]:
workers() 

1-element Vector{Int64}:
 2

In [9]:
rmprocs(2) # Para sacar procesos

Task (done) @0x00007f4c38d08a90

In [10]:
nprocs() 

1

In [11]:
workers()

1-element Vector{Int64}:
 1

---

In [12]:
function F(n)
    if n < 2
        return n
    else
        return F(n-1)+F(n-2)
    end
end

F (generic function with 1 method)

In [25]:
F(30)

832040

In [26]:
@time F(30);

  0.005672 seconds


In [22]:
import Base.Threads.@spawn

In [23]:
function fib(n::Int)
    if n < 2
        return n
    end
    t = @spawn fib(n - 2)
    return fib(n - 1) + fetch(t)
end

fib (generic function with 1 method)

In [29]:
fib(3)

2

In [33]:
@macroexpand @spawn fib(n - 2)

quote
    #= threadingconstructs.jl:172 =#
    let
        #= threadingconstructs.jl:173 =#
        local var"#61#task" = Base.Threads.Task((()->begin
                            #= threadingconstructs.jl:169 =#
                            fib(n - 2)
                        end))
        #= threadingconstructs.jl:174 =#
        (var"#61#task").sticky = false
        #= threadingconstructs.jl:175 =#
        if $(Expr(:islocal, Symbol("##sync#41")))
            #= threadingconstructs.jl:176 =#
            Base.Threads.put!(var"##sync#41", var"#61#task")
        end
        #= threadingconstructs.jl:178 =#
        Base.Threads.schedule(var"#61#task")
        #= threadingconstructs.jl:179 =#
        var"#61#task"
    end
end

## Datos mientras estudio concurrencia (borrar para la entrega final)


Task es una corutina es un hilo ligero que se comunica entre múltiples tareas a través del canal.

Es un tipo de dato que nos permite usar una unidad de trabajo computacional que podamos ejecutar cuando queramos y que se ejecute en su propia tarea (Se corre asinconicamente)

El canal puede verse como una tubería, legible en un extremo y grabable en un extremo.

- Se pueden pasar diferentes tareas **put!** Escribir simultáneamente en el mismo canal
- También se pueden pasar diferentes tareas **take!** Obtener datos simultáneamente del mismo canal

**Definición del canal**

Channel(func::Function, ctype=Any, csize=0, taskref=nothing)

Crea un **Channel** con un búfer interno que puede contener un máximo de *size* objetos de tipo *T*. 

**put!** invoca un bloque de canal completo hasta que se elimina un elemento con **take!**.

In [34]:
c1 = Channel(32)
c2 = Channel{Float64}(32)
put!(c1, 1)
put!(c2, 2)
take!(c2)

2.0

**Despues de cerrar el canal no se puede escribir más**

In [36]:
close(c1)
#put!(c1, 3) # debería romepr